<a href="https://colab.research.google.com/github/sanjaybora04/notebooks/blob/main/uptrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="text-align: center;">Retraining a Binary Classification Model by Identifying Data Drifts</h1>

In this example, we consider a binary classification task of human orientation while exercising. That is, given the location of 17 key-points of the body such as the nose, shoulders, wrist, hips, ankles, etc., the model tries to predict whether the person is in a horizontal (see image 1 below) or a vertical (see image 2 below) position. 

Input: 34-dimensional vector that contains the x and y positions of the 17 key-points.\
Output: Orientation (horizontal - class 0 or vertical - class 1)

![Horizontal_class.png](attachment:020c13d9-0eab-4a18-9d3a-fd645826319f.png)
![Vertical_class.png](attachment:5d35fb64-5f4b-4426-8756-2016a5ddc9a8.png)

In this notebook, we will see how we can use UpTrain package to identify data drift and identify out of distribution cases on real-world data.

### Install the required packages for this example

In [1]:
!pip install torch imgaug
!pip install uptrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 KB 2.8 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/uptrain-ai/uptrain.git
!mv uptrain/examples/1_orientation_classification/helper_files helper_files
!rm -rf uptrain

Cloning into 'uptrain'...
remote: Enumerating objects: 1405, done.
remote: Counting objects: 100% (544/544), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 1405 (delta 345), reused 350 (delta 233), pack-reused 861
Receiving objects: 100% (1405/1405), 2.68 MiB | 4.34 MiB/s, done.
Resolving deltas: 100% (779/779), done.


In [3]:
import os
import subprocess
import zipfile
import numpy as np
import torch
import uptrain

from helper_files import read_json, KpsDataset
from helper_files import body_length_signal, plot_all_cluster

Let's first download the example dataset

In [4]:
data_dir = "data"
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/data.zip"
orig_training_file = 'data/training_data.json'
if not os.path.exists(data_dir):
    print("Starting to download example dataset...")
    try:
        # Most Linux distributions have Wget installed by default.
        # Below command is to install wget for MacOS
        wget_installed_ok = subprocess.call("brew install wget", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
        print("Successfully installed wget")
    except:
        dummy = 1
    try:
        if not os.path.exists("data.zip"):
            file_downloaded_ok = subprocess.call("wget " + remote_url, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
            print("Data downloaded")
        with zipfile.ZipFile("data.zip", 'r') as zip_ref:
            zip_ref.extractall("./")
        print("Prepared Example Dataset")
        os.remove("data.zip")
    except Exception as e:
        print(e)
        print("Could not load training data")
        print("Please follow following steps to manually download data")
        print("Step 1: Paste the link https://oodles-dev-training-data.s3.amazonaws.com/data.zip in your browser")
        print("Step 2: Once the zip file is downloaded, unzip it here (i.e. YOUR_LOC/uptrain/examples/1_orientation_classification/")
else:
    print("Example dataset already present")

Starting to download example dataset...
Data downloaded
Prepared Example Dataset


#### Type of data files

Let's define the training and testing datasets, and visualise some of the training samples.

In [5]:
# Training data
training_file = 'data/training_data.json'

# A testing dataset to evaluate model performance
golden_testing_file = 'data/golden_testing_data.json'

# The data-points which the models sees in production
real_world_test_cases = 'data/real_world_testing_data.json'

# To annotate the collected data points, we extract the ground truth from a master annotation file 
# (we can also import from any other annotation pipelines such as an annotation job on Mechanical turk).
annotation_args = {'master_file': 'data/master_annotation_data.json'}

# Let's visualize some of the training examples
training_data = read_json(training_file, dataframe=True)
training_data.head()

,id,gt,Nose_X,Nose_Y,Left_Eye_X,Left_Eye_Y,Right_Eye_X,Right_Eye_Y,Left_Ear_X,Left_Ear_Y,...,Right_Hip_X,Right_Hip_Y,Left_Knee_X,Left_Knee_Y,Right_Knee_X,Right_Knee_Y,Left_Ankle_X,Left_Ankle_Y,Right_Ankle_X,Right_Ankle_Y
0,12002202098,1,536.832198,175.195562,275.183835,299.200492,321.438219,53.586762,438.831630,196.469141,...,355.320493,352.771307,282.603256,245.372110,-71.727270,148.854694,377.204408,118.812093,368.630415,32.683708
1,11004300163,1,127.299427,213.304615,120.006179,213.098156,119.749502,220.877945,123.240242,213.496218,...,282.739062,217.173672,331.887746,122.399542,369.934428,153.331971,407.764187,54.115847,373.738996,228.353284
2,18100208242,0,342.735809,75.173884,347.639763,69.066162,337.646757,69.463328,354.332650,70.183796,...,327.693125,175.678139,373.687116,236.403840,330.044019,238.719585,383.867725,289.895923,335.013380,292.018822
3,18100204340,0,335.262419,161.525842,340.358734,155.666068,329.570863,154.843509,347.034712,154.827196,...,316.727216,218.696415,375.741803,250.993843,285.306140,252.504808,384.364024,298.382275,275.523553,298.044853
4,12100104261,0,255.488160,62.442478,256.765451,58.603055,251.489867,58.323107,255.327531,55.634296,...,235.763698,136.902009,277.832912,177.679435,229.566259,191.052507,284.397434,213.477405,224.158806,239.154790


![2.jpg](attachment:27ccdf5e-4706-4ee9-bcce-c7a0c262f488.jpg)

### Step 1: Train our Neural Network model

We have defined a simple Neural Net comprised of a fully-connected layer with relu activation followed by a fully-connected layer to transfer latent features into model outputs. We compute Binary Entropy loss and are using Adam optimiser to train the model

![model.png](attachment:4d0b8efb-e059-4dad-8939-300423c98847.png)



In [6]:
from helper_files import get_accuracy_torch, train_model_torch, BinaryClassification
train_model_torch(training_file, 'version_0')

Training on:  data/training_data.json  which has  1000  data-points
Epoch 0: Loss 6.108189918821858
Epoch 1: Loss 1.5432152893753806
Epoch 2: Loss 0.9718629997024114
Epoch 3: Loss 0.7866231806232639
Epoch 4: Loss 0.56751997365115
Epoch 5: Loss 0.5176478891863001
Epoch 6: Loss 0.38865605023235894
Epoch 7: Loss 0.4058310522496875
Epoch 8: Loss 0.2512015529776268
Epoch 9: Loss 0.3784227873119747
Model saved at:  trained_models_torch/version_0


Next, let's see how our model performs on the golden testing dataset

In [7]:
get_accuracy_torch(golden_testing_file, 'version_0')

Evaluating model: version_0  on  15731  data-points


0.9289937066937893

Great, we have successfully trained our Neural Network which is giving more than 90% accuracy. We will now how we can use UpTrain package to identify data distribution shifts, collect edge cases and retrain the model to improve its accuracy 

### Step 2: Define the list of checks to perform on model

In this example, we define a simple data drift check to identify any distribution shift between real-world test set and the reference dataset (the training dataset in this case). To achieve this, we set 'kps' (Keypoints) as the input variable, the framework performs clustering on the training dataset and checks if the real-world test set is following the similar distribution

In [8]:
checks = [{
        'type': uptrain.Anomaly.DATA_DRIFT,
        'reference_dataset': orig_training_file,
        'is_embedding': True,
        "measurable_args": {
            'type': uptrain.MeasurableType.INPUT_FEATURE,
            'feature_name': 'kps'   # keypoints
        },
    }]

print("Data Drift checks: ", checks)

Data Drift checks:  [{'type': <Anomaly.DATA_DRIFT: 'data_drift'>, 'reference_dataset': 'data/training_data.json', 'is_embedding': True, 'measurable_args': {'type': <MeasurableType.INPUT_FEATURE: 'input_feature'>, 'feature_name': 'kps'}}]


### Step 3: Define the training and evaluation arguments

We now attach the model training and evaluation pipelines so that UpTrain framework can automatically retrain the model in case it sees that the model is facing significant data drift

In [9]:
# Define the training pipeline to annotate collected edge cases and retrain the model automatically
training_args = {
    "annotation_method": {"method": uptrain.AnnotationMethod.MASTER_FILE, "args": annotation_args}, 
    "training_func": train_model_torch, 
    "orig_training_file": orig_training_file,
}

# Define evaluation pipeline to test retrained model against original model
evaluation_args = {
    "inference_func": get_accuracy_torch,
    "golden_testing_dataset": golden_testing_file,
}

print("Training Pipelines: ", training_args, "\n")
print("Evaluation Pipelines: ", evaluation_args)

Training Pipelines:  {'annotation_method': {'method': <AnnotationMethod.MASTER_FILE: 1>, 'args': {'master_file': 'data/master_annotation_data.json'}}, 'training_func': <function train_model_torch at 0x7f49551069d0>, 'orig_training_file': 'data/training_data.json'} 

Evaluation Pipelines:  {'inference_func': <function get_accuracy_torch at 0x7f49551065e0>, 'golden_testing_dataset': 'data/golden_testing_data.json'}


### Step 4: Initialize the UpTrain Framework

In [10]:
cfg = {
    "checks": checks, 
    "training_args": training_args,
    "evaluation_args": evaluation_args,

    # Retrain when 200 datapoints are collected in the retraining dataset
    "retrain_after": 200,
    
    # A local folder to store the retraining dataset
    "retraining_folder": "uptrain_smart_data",
    
    # A function to visualize clusters in the data
    "cluster_visualize_func": plot_all_cluster,
}

# Initialize the UpTrain framework object with config 
framework = uptrain.Framework(cfg)
print("Successfully Initialized UpTrain Framework")

Successfully Initialized UpTrain Framework


### Step 5: Deploy the model in production

Ship the model to production worry-free because the UpTrain tool will identify any data drifts, collect interesting data points and automatically retrain the model on them. To mimic deployment behavior, we are running the model on a 'real-world test set' and logging model inputs with UpTrain framework.

In [11]:
inference_batch_size = 16
model_dir = 'trained_models_torch/'
model_save_name = 'version_0'
real_world_dataset = KpsDataset(
    real_world_test_cases, batch_size=inference_batch_size, is_test=True
)
model = BinaryClassification()
model.load_state_dict(torch.load(model_dir + model_save_name))
model.eval()

for i,elem in enumerate(real_world_dataset):

    # Do model prediction
    inputs = {"data": {"kps": elem[0]["kps"]}, "id": elem[0]["id"]}
    x_test = torch.tensor(inputs["data"]["kps"]).type(torch.float)
    test_logits = model(x_test).squeeze() 
    preds = torch.round(torch.sigmoid(test_logits)).detach().numpy()

    # Log model inputs and outputs to the uptrain Framework to monitor data drift
    idens = framework.log(inputs=inputs, outputs=preds)

    # Retrain only once
    if framework.version > 1:
        break

50  edge cases identified out of  13424  total samples
101  edge cases identified out of  15792  total samples
150  edge cases identified out of  24128  total samples
200  edge cases identified out of  26608  total samples

Kicking off re-training
Creating retraining dataset: uptrain_smart_data/1/training_dataset.json  by merging  data/training_data.json  and collected edge cases.

Training on:  uptrain_smart_data/1/training_dataset.json  which has  2005  data-points
Epoch 0: Loss 6.547668820619583
Epoch 1: Loss 2.09667319195981
Epoch 2: Loss 0.9921911590879351
Epoch 3: Loss 0.5552357899793435
Epoch 4: Loss 0.3871322651299773
Epoch 5: Loss 0.28291736927766714
Epoch 6: Loss 0.22658223121639368
Epoch 7: Loss 0.1783210203748492
Epoch 8: Loss 0.11531238443450138
Epoch 9: Loss 0.13162718854009495
Model saved at:  trained_models_torch/version_1
Model retraining done...

Generating comparison report...
Evaluating model: version_0  on  15731  data-points
Evaluating model: version_1  on  15731 

#### Hurray! Our model after retraining performs significantly better.

Let's try to understand how UpTrain helped to improve our classification model.

#### Training data clusters
While initializing the UpTrain framework, it clusters the reference dataset (i.e. training dataset in our case). We are plotting the centroids and support (ie number of data-points belonging to that cluster) of all the 20 clusters in our training dataset.

![training_dataset_clusters.png](attachment:56779bcd-7f8b-4e7a-8af3-c87a83bb0883.png)

#### Edge cases clusters
As we see, the UpTrain framework identifies out-of-distribution data-points and collects the edge-cases which are sparsely present in the training dataset.

![collected_edge_cases_clusters.png](attachment:5a57b1ed-6cff-4482-a60a-b04bba3c40a5.png)

From the above plot generated while monitoring the model in production, we see that data drift occurs for many cases when the person is in a horizontal position. Specifically, cases when the person is in push-ups position are very sparse in our training dataset, causing the model predictions to go wrong for them. In the example of edge-case detection, we will see that how we can use this insight to define a "Pushup" signal, collect all push-up related data-points and specifically retrain on them.

## Do more with UpTrain

Apart from data drift, UpTrain has many other features such as 
1. Checking for edge-cases and collecting them for automated retraining
2. Verifying data integrity, 
3. Monitoring model performance and accuracy of predictions, etc. 

To dig deeper into it, we recommend you checkout the other examples in the folder "deepdive_examples".